In [1]:
import math
import json

import rasterio
import rasterio.plot
from rasterio.mask import mask 

from osgeo import gdal
gdal.UseExceptions()
    
import geopandas as gpd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt

from rasterio.plot import plotting_extent
import earthpy as et
import earthpy.spatial as es
import earthpy.plot as ep

import os

In [2]:
def get_bbox(geom):
    pairs = []

    for i in range(0, len(geom)-1, 2):
        pairs.append((geom[i], geom[i+1]))

    x = []
    y = []
    for i in pairs:
        x.append(i[0])
        y.append(i[1])

    bbox = [min(x), min(y), round(max(x)-min(x),2), round(max(y)-min(y),2)]
    
    return bbox

### Abrir arquivos

- Ortomosaico
- Grade
- Vetor das copas

In [3]:
img = rasterio.open('../IMGS/odm_orthophoto.tif')

grid = gpd.read_file('../VECTOR/grid_from_img.geojson').set_index('id')
grid_2 = gpd.read_file('../VECTOR/grid_from_img-2.geojson').set_index('id')

copas = gpd.read_file('../VECTOR/copas.geojson')

In [4]:
info = {
        "description": "Felipe Sa 2021 - HLB",
        "url": "http://siteaqui.com",
        "version": "1.0",
        "year": 2021,
        "contributor": "Felipe Sa",
        "date_created": "2021/01/01"
        }

licenses = [
                {
                "url": "http://creativecommons.org/licenses/by-nc-sa/2.0/",
                "id": 1,
                "name": "Attribution-NonCommercial-ShareAlike License"
                },
                {
                "url": "http://creativecommons.org/licenses/by-nc/2.0/",
                "id": 2,
                "name": "Attribution-NonCommercial License"
                }
            ]

In [7]:
files = [int(x[6:].split('.')[0]) for x in os.listdir('../VECTOR/CORTES/EDITADOS')]
files.sort()

# Filtrar so valores de grid_2
# files = [x for x in files if x in grid_2.index]

images = []

for i in files:
    
    img = rasterio.open('../IMGS/CORTES/corte_{}.tif'.format(i))
    
    
    images.append({ "license": 1,
                    "file_name": "{:04}.tif".format(i),
                    "coco_url": "empty",
                    "height": img.height,
                    "width": img.width,
                    "date_captured": "2020-01-01 00:00:00",
                    "flickr_url": "empty",
                    "id": i
                    })

In [10]:
[(x[6:].split('.')[0]) for x in os.listdir('../VECTOR/CORTES/EDITADOS')]

['459',
 '233',
 '286',
 '410',
 '184',
 '123',
 '386',
 '127',
 '183',
 '401',
 '85',
 '261',
 '361',
 '280',
 '421',
 '343',
 '435',
 '390',
 '425',
 '387',
 '255',
 '158',
 '320',
 '268',
 '239',
 '445',
 '338',
 '87',
 '182',
 '281',
 '328',
 '429',
 '73',
 '103',
 '416',
 '454',
 '368',
 '340',
 '384',
 '382',
 '126',
 '162',
 '462',
 '161',
 '404',
 '168',
 '415',
 '160',
 '145',
 '181',
 '74',
 '146',
 '385',
 '440',
 '350',
 '419',
 '461',
 '388',
 '232',
 '187',
 '290',
 '318',
 '304',
 '377',
 '190',
 '441',
 '102',
 '238',
 '469',
 '140',
 '359',
 '428',
 '240',
 '356',
 '236',
 '327',
 '159',
 '309',
 '265',
 '420',
 '208',
 '62',
 '279',
 '295',
 '366',
 '351',
 '457',
 '360',
 '412',
 '253',
 '230',
 '379',
 '337',
 '403',
 '376',
 '331',
 '312',
 '355',
 '61',
 '106',
 '256',
 '180',
 '378',
 '341',
 '317',
 '353',
 '357',
 '75',
 '278',
 '406',
 '326',
 '439',
 '147',
 '237',
 '349',
 '444',
 '148',
 '443',
 '263',
 '241',
 '467',
 '315',
 '374',
 '394',
 '362',
 '262',

In [13]:
categories = [{"supercategory": "canopy","id": 1,"name": "hamlin"}]

In [ ]:
files = [int(x[6:-8]) for x in os.listdir('../VECTOR/CORTES/EDITADOS')]
files.sort()

annotations = []

for i in files:
    
    copas = gpd.read_file('../VECTOR/CORTES/EDITADOS/corte_{}.geojson'.format(i))
    
    for row in copas.index:
        
        geom_px = [float(x) for x in copas.loc[row, 'geometry_image'].split(', ')]
        
        
        annotations.append({"segmentation": [geom_px],
                            "area": copas.loc[row,'geometry'].area/0.0025,
                            "iscrowd": 0,
                            "image_id": i,
                            "bbox": get_bbox(geom_px),
                            "category_id": 1,
                            "id": int(str(i) + '0' + str(row + 1)) # Não definido ainda
                        })
    
        area = copas.loc[row, 'geometry'].area/0.0025


In [ ]:
annotation = {
                "info": info,
                "licenses": licenses,
                "images": images,
                "categories": categories,
                "annotations": annotations,# <-- Not in Captions annotations
#                 "segment_info": []  # <-- Only in Panoptic annotations
                }

In [ ]:
with open('../ANNOTATIONS/grid_2.json', 'w') as outfile:
    json.dump(annotation, outfile)